In [54]:
import pandas as pd

csv_data = pd.read_csv('./data/csv_data.csv', index_col=0)  
csv_data.head(10)

,time,code,next_close,close,pe_ratio,pb_ratio,pcf_ratio,market_cap,PEG,roe_ttm,...,sharpe_ratio_20,net_profit_growth_rate,profit_margin_ttm,roa_ttm,net_operate_cashflow_growth_rate,np_parent_company_owners_growth_rate,total_asset_growth_rate,operating_revenue_growth_rate,pct_change,target
0,2010-12-31,000002.XSHE,6.70,6.20,-0.353477,-0.353307,0.360895,903.8063,-0.352715,-0.352921,...,0.353005,-0.353861,-0.352261,-0.353304,-0.371200,-0.353819,-0.351791,-0.354569,0.080645,1
1,2010-12-31,000006.XSHE,3.46,3.19,0.226630,0.226844,-0.220561,53.0260,0.226471,0.227204,...,-0.227522,0.228176,0.227481,0.226824,0.215443,0.228131,0.226249,0.229046,0.084639,1
2,2010-12-31,000009.XSHE,6.97,7.34,0.138608,0.138669,-0.136737,182.9189,0.138135,0.137837,...,-0.138064,0.137460,0.138245,0.137542,0.114731,0.137840,0.138549,0.136940,-0.050409,-1
3,2010-12-31,000012.XSHE,10.70,11.11,-0.016728,-0.015922,0.045756,410.1524,-0.017483,-0.016126,...,0.015786,-0.015464,-0.015792,-0.015795,-0.007689,-0.015469,-0.017394,-0.015739,-0.036904,-1
5,2010-12-31,000024.XSHE,11.16,9.96,0.075874,0.075997,-0.068320,273.9094,0.076690,0.076474,...,-0.076295,0.076112,0.077632,0.076371,0.060943,0.075618,0.076809,0.078440,0.120482,1
6,2010-12-31,000028.XSHE,29.26,31.09,0.197032,0.198329,-0.191321,95.0893,0.197717,0.197791,...,-0.197563,0.196739,0.195676,0.196904,0.206943,0.196697,0.197055,0.196540,-0.058861,-1
7,2010-12-31,000029.XSHE,4.90,4.66,0.231433,0.230854,-0.214374,47.5480,0.230228,0.230467,...,-0.231228,0.231911,0.230762,0.230408,0.219044,0.231866,0.229652,0.230977,0.051502,1
8,2010-12-31,000031.XSHE,5.96,5.67,0.186061,0.185874,-0.184684,112.9955,0.185763,0.185899,...,-0.186782,0.186419,0.187152,0.185770,0.223000,0.186644,0.187945,0.188157,0.051146,1
10,2010-12-31,000042.XSHE,8.01,7.53,0.234295,0.234500,-0.219109,41.5468,0.234642,0.235065,...,-0.235015,0.234291,0.236470,0.234824,0.230902,0.234268,0.233869,0.232661,0.063745,1
11,2010-12-31,000046.XSHE,3.95,3.71,0.124357,0.123504,-0.124028,204.6233,0.123997,0.123229,...,-0.123328,0.123563,0.124494,0.123182,0.160471,0.123469,0.122715,0.124449,0.064690,1


In [55]:
## Target and Label
features = list(set(csv_data.columns) - set(['pct_change', 'close', 'target', 'time', 'code', 'next_close']))
labels = csv_data[features]
target = csv_data.target

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, r2_score
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE, SelectKBest, chi2, f_regression
from xgboost import XGBClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC

## 训练数据和验证数据
X_train, X_test, y_train, y_test = train_test_split(labels, target, test_size=0.33)

### Feature Selection


In [ ]:
pipeline = Pipeline([
    ('sclaer', StandardScaler()),
    ('rfecv', RFE(RandomForestClassifier(random_state=40))),
    ('classifier', XGBClassifier())
])


abc = GridSearchCV(
    estimator=pipeline,
    param_grid={
        'classifier__n_estimators': (100, 110, 120),
        'classifier__max_depth': (3, 4, 5, 6),
        'classifier__learning_rate': (0.1, 0.2, 0.3)
    }, 
    cv=4,
    n_jobs=4)

abc.fit(X_train, y_train)

In [ ]:
y_pred = abc.predict(X_test)
print('分类报告: %s' % classification_report(y_test, y_pred))

In [31]:
abc.fit(labels, target)

def save_model(m_model, name):
    import pickle
    from six import BytesIO
    pkl_file = open(f'{name}.pkl', 'wb')
    pickle.dump(m_model, pkl_file, 0)
    pkl_file.close()
    
save_model(abc, 'svm')

KeyboardInterrupt: 